Review Analysis using   --- TFIDF

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
data1= pd.read_csv('/content/drive/MyDrive/Reviews.csv')

In [3]:
list=data1.columns 


In [4]:
data1['HelpPercentage'] =np.where(data1['HelpfulnessDenominator']>0,data1['HelpfulnessNumerator']/data1['HelpfulnessDenominator'],-1)

In [5]:
data1['UpVotePercent']=pd.cut(data1['HelpPercentage'],bins =[-1,0,0.2,0.4,0.6,0.8,1],labels=['Empty','0-20%','20-40','40-60','60-80','80-100'])

In [6]:
data2 = data1[data1['Score']!=3]
data2['Score'].unique()

array([5, 1, 4, 2])

In [7]:
X = data2['Text']

In [8]:
y_dictionary ={1:0,2:0,5:1,4:1}
Y = data2['Score'].map(y_dictionary)


In [9]:
def txt_fit(X,Y,nlp_mdl,ml_mdl,Show_value=False):
  X_Cv = nlp_mdl.fit_transform(X)
  print('features:{}'.format(X_Cv.shape[1]))
  X_train,X_test, Y_train , Y_test = train_test_split(X_Cv,Y)
  m= ml_mdl.fit(X_train,Y_train)
  accuracy =m.score(X_test,Y_test)
  print(accuracy)

  if Show_value == True:
    w = cvc.get_feature_names()
    Coefficent = m.coef_.tolist()[0]
    Coeff_Data = pd.DataFrame({'Word':w ,'Coefficient':Coefficent})
    Coeff_Data = Coeff_Data.sort_values(['Coefficient','Word'],ascending=False )
    print("\n")
    print("Top 10 Negative words")
    print(Coeff_Data.tail(10))
  


In [10]:
cvc = CountVectorizer(stop_words='english')

#txt_fit(X,Y,cvc,LogisticRegression(),True) 


In [11]:
from sklearn.metrics import confusion_matrix,accuracy_score
def pred(X,Y,nlp_mdl,ml_mdl):
  X_Cv = nlp_mdl.fit_transform(X)
  X_train,X_test, Y_train , Y_test = train_test_split(X_Cv,Y)
  m= ml_mdl.fit(X_train,Y_train)
  predctns = m.predict(X_test)
  cmtrx = confusion_matrix(predctns,Y_test)

  print(cmtrx)
  accscore = accuracy_score(predctns,Y_test)

  print(accscore)

In [12]:
pred(X,Y,CountVectorizer(stop_words='english'),LogisticRegression())

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[ 14843   2854]
 [  5493 108264]]
0.9365025027766367


In [13]:
from sklearn.dummy import DummyClassifier


In [14]:
#txt_fit(X,Y,cvc,DummyClassifier(),False)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [16]:
tfidfV = TfidfVectorizer(stop_words='english')

In [17]:
#txt_fit(X,Y,tfidfV,LogisticRegression(), True)

In [18]:
#pred(X,Y,cvc,LogisticRegression())

In [19]:
#pred(X,Y,tfidfV,LogisticRegression())

In [20]:
data = data1[data1['Score']==5]

In [21]:
data['UpVotePercent'].unique()

['80-100', NaN, '60-80', 'Empty', '40-60', '20-40', '0-20%']
Categories (6, object): ['Empty' < '0-20%' < '20-40' < '40-60' < '60-80' < '80-100']

In [22]:
data2 = data[data['UpVotePercent'].isin(['80-100','60-80', '20-40', '0-20%'])]

In [23]:
X = data2['Text']

In [24]:
X

0         I have bought several of the Vitality canned d...
8         Right now I'm mostly just sprouting this so my...
10        I don't know if it's the cactus or the tequila...
11        One of my boys needed to lose some weight and ...
14        The Strawberry Twizzlers are my guilty pleasur...
                                ...                        
568440    Indie Candy's gummies are absolutely delicious...
568444    As a foodie, I use a lot of Chinese 5 Spice po...
568445    You can make this mix yourself, but the Star A...
568451    These stars are small, so you can give 10-15 o...
568452    These are the BEST treats for training and rew...
Name: Text, Length: 154428, dtype: object

In [25]:
data2['UpVotePercent'].unique()

['80-100', '60-80', '20-40', '0-20%']
Categories (4, object): ['0-20%' < '20-40' < '60-80' < '80-100']

In [26]:
y_dictionary = {'80-100':1, '60-80':1,'20-40':0, '0-20%':0}
Y = data2['UpVotePercent'].map(y_dictionary)

In [27]:
Y

0         1.0
8         1.0
10        1.0
11        1.0
14        1.0
         ... 
568440    1.0
568444    1.0
568445    1.0
568451    1.0
568452    1.0
Name: UpVotePercent, Length: 154428, dtype: float64

In [28]:
Y.value_counts()

1.0    151721
0.0      2707
Name: UpVotePercent, dtype: int64

In [29]:
tfidf = TfidfVectorizer()

In [30]:
X_Cv = tfidf.fit_transform(X)

In [31]:
X_Cv

<154428x67507 sparse matrix of type '<class 'numpy.float64'>'
	with 8919285 stored elements in Compressed Sparse Row format>

In [32]:
from imblearn.over_sampling import RandomOverSampler

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [33]:
ros = RandomOverSampler()

In [34]:
X_train_rsamp,Y_train_rsamp =ros.fit_sample(X_Cv,Y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [35]:
X_train_rsamp.shape

(303442, 67507)

In [36]:
Y_train_rsamp.shape

(303442,)

In [37]:
from collections import Counter


In [38]:
#Resampled data 0.0 & 1.1 are balanced
cn =Counter(Y_train_rsamp)

cn

Counter({0.0: 151721, 1.0: 151721})

In [39]:
#Original Data
cn = Counter(Y)
cn

Counter({0.0: 2707, 1.0: 151721})

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [69]:
lg_cl = LogisticRegression()

grid = {'C':10.0**np.arange(-2,3),'penalty':['l1','l2'] }

In [70]:
clf = GridSearchCV(estimator=lg_cl,param_grid=grid,cv=5,n_jobs=-1,scoring='f1_macro')

In [71]:
clf.fit(X_train_rsamp,Y_train_rsamp)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [73]:
lg_cl.get_params().keys()


dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [75]:
X_train,X_test,Y_train,Y_test = train_test_split(X_Cv,Y)

In [77]:
predictn = clf.predict(X_test)

In [78]:
predictn

array([1., 1., 0., ..., 1., 1., 1.])

In [79]:
confusion_matrix(Y_test,predictn)

array([[  667,     0],
       [  491, 37449]])

In [80]:
accuracy_score(Y_test,predictn)
#Using Cross Validation GridCV we got more accurate model here.

0.9872820991011992

In [86]:
clf.best_params_


{'C': 100.0, 'penalty': 'l2'}

In [90]:
clf.score

<bound method BaseSearchCV.score of GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)>

In [91]:
clf.predict_proba

<function sklearn.model_selection._search.BaseSearchCV.predict_proba>